# 데이터 전처리 작업 및 데이터 저장

이 코드는 이미지 데이터의 데이터 전처리를 수행하고, 결과를 CSV 파일로 저장하는 작업을 수행합니다.   
데이터 전처리 과정은 얼굴을 감지하고 잘라내며, 이미지를 그레이스케일로 변환하여 저장됩니다. 

In [1]:
# !pip install tqdm

In [2]:
import os
from glob import glob
import json
import cv2
import numpy as np
import pandas as pd
from random import shuffle
from tqdm import tqdm
from functools import partial
CROP_SIZE = (224,224) #사진크기 지정


In [3]:
def create_path(path):
    if not os.path.exists(path):
        os.mkdir(path)
        print(f'Create {path}.')
        return path
    else:
        print(f'{path} already exists.')
        return path

In [4]:
class Datapreprocess:
    def __init__(self,emo_pic_path):
        """
        데이터 전처리 클래스의 초기화 메서드입니다.
        """
        self.img_folder_path = emo_pic_path
        self.data_lst = []  # 데이터 목록을 저장할 리스트
        self.img = None  # 이미지 데이터를 저장할 변수
        self.X_position = (0, 0)  # 얼굴 영역의 X 좌표 범위
        self.Y_position = (0, 0)  # 얼굴 영역의 Y 좌표 범위
        self.curr_emotion = ''  # 현재 감정 레이블
        self.new_file_path = ""  # 새로운 파일 경로
        self.img_path = ""
        self.file_name = ""
        self.curr_dir = os.getcwd()
        
    def _save_crop_img(self):
        img = self.img.copy()
        roi = img[
            int(self.Y_position[0]):int(self.Y_position[1]),
            int(self.X_position[0]):int(self.X_position[1]),
        ]
        img = cv2.resize(roi, CROP_SIZE, interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        self.img = img

    def _detect_face(self,info):
        self.img_path = os.path.join(self.img_folder_path,self.file_name)
        self.img = cv2.imread(self.img_path)
        self.X_position = info['annot_A']['boxes']['minX'], info['annot_A']['boxes']['maxX']
        self.Y_position = info['annot_A']['boxes']['minY'], info['annot_A']['boxes']['maxY']


    def work(self, info, emo):
        """
        데이터 전처리 작업을 수행하는 메서드입니다.

        :param img_path: 입력 이미지 파일의 경로
        :param emo: 해당 이미지의 감정 레이블
        :return: 데이터 전처리 성공 여부 (True 또는 False)
        """
        self.curr_emotion = emo
        self.file_name = info['filename']
        self._detect_face(info)
        self._save_crop_img()
        self.new_file_path = self._get_new_file_path(self.file_name)
        self.data_lst.append({"path": self.new_file_path, "label": self.curr_emotion})
        cv2.imwrite(self.new_file_path, self.img)
        df = pd.DataFrame(self.data_lst)
        df.to_csv(f"./validation/csvs/{self.curr_emotion}.csv")
        
    def _get_new_file_path(self,file_name):
        return f"./validation/{self.curr_emotion}/{file_name}"

In [ ]:
emotion_lst = ['angry','embarrassed','happy','sad','neutral']


datapreprocess = Datapreprocess(None)
curr_dir = os.getcwd()
create_path(os.path.join(curr_dir,'validation','csvs'))
errors_path = create_path(os.path.join(curr_dir,'validation','errors'))
label_path = 'D:\ML_pj\Validation\labels'
pic_path = "D:\ML_pj\Validation"

for emo in emotion_lst:
    emo_pic_path = create_path(os.path.join(pic_path,emo))
    create_path(os.path.join(curr_dir,'validation',emo))
    datapreprocess.__init__(emo_pic_path)
    json_path = os.path.join(label_path,f"new_{emo}.json")
    error_path = os.path.join(errors_path,f'{emo}_error.txt')
    with open(json_path,'r') as f:
        info_lst = json.load(f)
    for info in tqdm(info_lst,desc = emo):
        try:
            datapreprocess.work(info, emo)  # 각 이미지에 대해 데이터 전처리 작업을 수행합니다.
        except:
            with open(error_path,'a')as f:
               f.write(str(info['filename'])+'\n') 

C:\Users\royal\Desktop\ML\project\validation\csvs already exists.
C:\Users\royal\Desktop\ML\project\validation\errors already exists.
D:\ML_pj\Validation\angry already exists.
C:\Users\royal\Desktop\ML\project\validation\angry already exists.


angry: 100%|███████████████████████████████████████████████████████████████████████| 7461/7461 [22:11<00:00,  5.61it/s]


D:\ML_pj\Validation\embarrassed already exists.
C:\Users\royal\Desktop\ML\project\validation\embarrassed already exists.


embarrassed: 100%|█████████████████████████████████████████████████████████████████| 7454/7454 [16:49<00:00,  7.39it/s]


D:\ML_pj\Validation\happy already exists.
Create C:\Users\royal\Desktop\ML\project\validation\happy.


happy: 100%|███████████████████████████████████████████████████████████████████████| 7499/7499 [26:41<00:00,  4.68it/s]


D:\ML_pj\Validation\sad already exists.
Create C:\Users\royal\Desktop\ML\project\validation\sad.


sad: 100%|█████████████████████████████████████████████████████████████████████████| 7479/7479 [26:10<00:00,  4.76it/s]


D:\ML_pj\Validation\neutral already exists.
Create C:\Users\royal\Desktop\ML\project\validation\neutral.


neutral:  89%|█████████████████████████████████████████████████████████████▎       | 6576/7403 [22:21<02:23,  5.78it/s]